In [3]:
# Importamos librerías
import pandas as pd
import json
from langdetect import detect
import nltk
import numpy as np
from nltk.sentiment import SentimentIntensityAnalyzer

In [3]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Hp\AppData\Roaming\nltk_data...


True

In [8]:
ruta = 'australian_user_reviewsR.json'
with open(ruta, 'r', encoding='utf-8') as f:
    data = json.load(f)  
df_UserReviews = pd.DataFrame(data)

In [10]:
# Mostrar las primeras filas del DataFrame
print("Primeras filas del DataFrame:")
df_UserReviews.head()

Primeras filas del DataFrame:


,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


In [12]:
with open('australian_user_reviewsR.json', 'r', encoding='utf-8') as file:
    URev = json.loads(file.read())

In [15]:
# Calcula valores en la columna 'recommend' 
conteo_por_recomendaciones = df_UserReviews['recommend'].value_counts()
porcentaje_recomendaciones = df_UserReviews['recommend'].value_counts(normalize=True) * 100
resumen_recomendaciones = pd.DataFrame({
    'Conteo': conteo_por_recomendaciones,
    'Porcentaje': porcentaje_recomendaciones.round(2).astype(str) + '%'
})

resumen_recomendaciones = resumen_recomendaciones.sort_values(by='Conteo', ascending=False)
print(resumen_recomendaciones)

           Conteo Porcentaje
recommend                   
True        52473     88.48%
False        6832     11.52%


In [16]:

# Reemplazar valores vacíos, 'null' y 'None' con NaN en todo el DataFrame
df_UserReviews.replace(['', 'null', 'None'], np.nan, inplace=True)

In [17]:

# nulos
columnas_a_considerar = ['posted', 'recommend', 'review']
filas_con_nulos = df_UserReviews[df_UserReviews[columnas_a_considerar].isnull().all(axis=1)]
filas_con_nulos

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review


In [22]:

# Se establece umbral del 80% para decidir que columnas eliminar por valores nulos
umbral_nulos = 0.8

# Calcula el porcentaje de valores nulos por columna
porcentaje_nulos = df_UserReviews.isnull().mean()
columnas_a_eliminar = porcentaje_nulos[porcentaje_nulos > umbral_nulos]
print("Columnas con más del {}% de valores nulos (candidatas a eliminar):".format(umbral_nulos * 100))
for columna, porcentaje in columnas_a_eliminar.items():
    print("{}: {:.2%}".format(columna, porcentaje))

Columnas con más del 80.0% de valores nulos (candidatas a eliminar):


In [23]:

# Eliminamos las columnas que pasan el umbral establecido para valores nulos 
df_UserReviews.drop(columns=columnas_a_eliminar.index, inplace=True)
df_UserReviews.sort_values('user_id')
filas_duplicadas = df_UserReviews[df_UserReviews.duplicated(subset=['user_id', 'item_id', 'posted', 'review'], keep=False)]
filas_duplicadas.count()

user_id      1734
user_url     1734
posted       1734
item_id      1734
helpful      1734
recommend    1734
review       1734
dtype: int64

In [24]:
# Se eliminan los registros duplicados
duplicados_eliminados = df_UserReviews.drop_duplicates(subset=['user_id', 'item_id', 'posted', 'review'], keep=False, inplace=True)
cantidad_total_duplicados_eliminados = filas_duplicadas.shape[0]
print("Cantidad total de registros duplicados eliminados:", cantidad_total_duplicados_eliminados)

Cantidad total de registros duplicados eliminados: 1734


In [26]:
unique_posted_values = df_UserReviews['posted'].unique()
i = 0
for value in unique_posted_values:
    if i <= 10:
        print(value)
        i += 1
    else: break

Posted November 5, 2011.
Posted July 15, 2011.
Posted April 21, 2011.
Posted June 24, 2014.
Posted September 8, 2013.
Posted November 29, 2013.
Posted February 3.
Posted December 4, 2015.
Posted November 3, 2014.
Posted October 15, 2014.
Posted October 14, 2013.


In [27]:
df_UserReviews['posted'] = pd.to_datetime(df_UserReviews['posted'].astype(str).str.replace(r'Posted |,|\.', '', regex=True), errors='coerce')
# Crea la columna 'year' a partir de 'posted'
df_UserReviews['year'] = df_UserReviews['posted'].dt.year.astype('Int64')
df_UserReviews = df_UserReviews.sort_values(['item_id', 'year'])

In [28]:
df_UserReviews['year'] = df_UserReviews.groupby('item_id', group_keys=False)['year'].apply(lambda group: group.interpolate(method='pad') if group.notna().any() else group)
df_UserReviews['year'] = df_UserReviews['year'].fillna(df_UserReviews['year'].median())

C:\Users\Hp\AppData\Local\Temp\ipykernel_19332\2234249454.py:2: FutureWarning: Series.interpolate with method=pad is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_UserReviews['year'] = df_UserReviews.groupby('item_id', group_keys=False)['year'].apply(lambda group: group.interpolate(method='pad') if group.notna().any() else group)


In [29]:
# Se eliminan las columnas 'posted' y 'user_id' que ya no son de utilidad
df_UserReviews.drop(['posted', 'user_id'], axis=1, inplace=True)
df_UserReviews.head()

,user_url,item_id,helpful,recommend,review,year
5331,http://steamcommunity.com/profiles/76561198040...,10,0 of 2 people (0%) found this review helpful,True,this game is the 1# online action game is awes...,2011
22702,http://steamcommunity.com/id/maddoxx789,10,No ratings yet,True,GYERTEK GAMELNI MINDENKI ITT VAN AKI SZÁMIT !!...,2012
35539,http://steamcommunity.com/id/mixadance,10,2 of 3 people (67%) found this review helpful,True,:D,2012
43134,http://steamcommunity.com/id/waspish,10,No ratings yet,True,Good Game :D,2012
24137,http://steamcommunity.com/profiles/76561198001...,10,No ratings yet,True,jueguenlooooooo,2013


In [30]:
# Análisis de idioma en reseñas

def detectar_idioma(texto):
    try:
        return detect(texto)
    except:
        return None
    
df_UserReviews['language'] = df_UserReviews['review'].apply(detectar_idioma)

# Calcular el conteo y porcentaje de cada idioma
conteo_por_idioma = df_UserReviews['language'].value_counts()
porcentaje_por_idioma = df_UserReviews['language'].value_counts(normalize=True) * 100
resumen_idiomas = pd.DataFrame({
    'Conteo': conteo_por_idioma,
    'Porcentaje': porcentaje_por_idioma.round(2).astype(str) + '%'
})

resumen_idiomas = resumen_idiomas.sort_values(by='Conteo', ascending=False)
resumen_idiomas.head()

,Conteo,Porcentaje
language,,
en,44382,77.84%
pt,2131,3.74%
es,1245,2.18%
de,1123,1.97%
so,993,1.74%


In [31]:
mapeo_idiomas = {
    'en': 'English',
    'pt': 'Portugués',
    'es': 'Español',
    'de': 'German',
    'so': 'Somali',
}

# Aplicar el mapeo al DataFrame
df_UserReviews['idioma_completo'] = df_UserReviews['language'].map(mapeo_idiomas)
resumen_idiomas = df_UserReviews['idioma_completo'].value_counts().reset_index()
resumen_idiomas.columns = ['Idioma', 'Conteo']
resumen_idiomas['Conteo'] = resumen_idiomas['Conteo'].round(2)
resumen_idiomas['Porcentaje'] = (resumen_idiomas['Conteo'] / len(df_UserReviews)) * 100
resumen_idiomas['Porcentaje'] = resumen_idiomas['Porcentaje'].round(2)
# Guardar en un archivo CSV
resumen_idiomas.to_csv('resumen_idiomas.csv', index=False)

In [32]:
# Filtrar los registros donde no es igual a en 
df_UserReviews = df_UserReviews[df_UserReviews['idioma_completo'] == 'English']
df_UserReviews = df_UserReviews.drop('idioma_completo', axis=1)

In [33]:
df_UserReviews.loc[:, 'review'] = df_UserReviews['review'].str.lower()
#Eliminación de caracteres especiales
df_UserReviews['review'] = df_UserReviews['review'].replace('[^A-Za-z0-9\s]+', '', regex=True)
df_UserReviews.loc[:, 'review'] = df_UserReviews['review'].str.replace('[^\w\s]', '', regex=True)

In [35]:
# Análisis de sentimiento
def analyze_sentiments(df):
    sia = SentimentIntensityAnalyzer()
    df['compound_score'] = df['review'].apply(lambda review: sia.polarity_scores(review)['compound'])
    df['sentiment_analysis'] = df['compound_score'].apply(lambda score: 0 if score < 0 else (1 if score == 0 else 2))
    score_counts = df['sentiment_analysis'].value_counts()
    blank_reviews_count = df['review'].isnull().sum()
    total_reviews = len(df)
    score_percentages = (score_counts / total_reviews * 100).round(2)
    blank_reviews_percentage = (blank_reviews_count / total_reviews * 100).round(2)
    df.drop(['review','compound_score'], axis=1, inplace=True)  

    return df, score_counts, blank_reviews_count, score_percentages, blank_reviews_percentage

In [36]:
# Llamar a la función analyze_sentiments
df_UserReviews, score_counts, blank_reviews_count, score_percentages, blank_reviews_percentage = analyze_sentiments(df_UserReviews)

resumen_sentimientos = pd.DataFrame({
    'Conteo': score_counts,
    'Porcentaje': score_percentages.round(2).astype(str) + '%'
})
resumen_sentimientos = resumen_sentimientos.sort_values(by='Conteo', ascending=False)
print("\nResumen de análisis de sentimientos:")
print(resumen_sentimientos)
print("\nConteo de reviews en blanco: ", blank_reviews_count, " Porcentaje: ", blank_reviews_percentage.round(2).astype(str) + '%')


Resumen de análisis de sentimientos:
                    Conteo Porcentaje
sentiment_analysis                   
2                    31451     70.86%
0                     8276     18.65%
1                     4655     10.49%

Conteo de reviews en blanco:  0  Porcentaje:  0.0%


In [38]:
df_UserReviews = df_UserReviews.drop('language', axis=1)
df_UserReviews.to_csv('user_reviews_cleaned.csv', index=False)